<a href="https://colab.research.google.com/github/ryuseunghan/IssueTracker/blob/main/title_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install konlpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 73.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.3/465.3 kB 39.5 MB/s eta 0:00:00


In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
def preprocessing(text):
    # 문제를 일으킬 수 있는 문자 제거
    bad_chars = {"\u200b": "", "…": " ... ", "\ufeff": ""}
    for bad_char in bad_chars:
        text = text.replace(bad_char, bad_chars[bad_char])

    error_chars = {"\u3000": " ", "\u2009": " ", "\u2002": " ", "\xa0":" "}
    for error_char in error_chars:
        text = text.replace(error_char, error_chars[error_char])

    # 이메일 제거
    text = re.sub(r"[a-zA-Z0-9+-_.]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+", "[이메일]", text).strip()

    # "#문자" 형식 어절 제거
    text = re.sub(r"#\S+", "", text).strip()

    # "@문자" 형식 어절 제거
    text = re.sub(r"@\w+", "", text).strip()

    # URL 제거
    text = re.sub(r"(http|https)?:\/\/\S+\b|www\.(\w+\.)+\S*", "[웹주소]", text).strip()
    text = re.sub(r"pic\.(\w+\.)+\S*", "[웹주소]", text).strip()

    # 뉴스 저작권 관련 텍스트 제거
    re_patterns = [
        r"\<저작권자(\(c\)|ⓒ|©|\(Copyright\)|(\(c\))|(\(C\))).+?\>",
        r"저작권자\(c\)|ⓒ|©|(Copyright)|(\(c\))|(\(C\))"
    ]

    for re_pattern in re_patterns:
        text = re.sub(re_pattern, "", text).strip()

    # 뉴스 내 포함된 이미지에 대한 레이블 제거
    text = re.sub(r"\(출처 ?= ?.+\) |\(사진 ?= ?.+\) |\(자료 ?= ?.+\)| \(자료사진\) |사진=.+기자 ", "", text).strip()

    # 중복 문자 처리
    text = repeat_normalize(text, num_repeats=2).strip()

    # 문제를 일으킬 수 있는 구두점 치환
    punct_mapping = {"‘": "'", "₹": "e", "´": "'", "°": "", "€": "e", "™": "tm", "√": " sqrt ", "×": "x", "²": "2", "—": "-", "–": "-", "’": "'", "_": "-", "`": "'", '“': '"', '”': '"', '“': '"', "£": "e", '∞': 'infinity', 'θ': 'theta', '÷': '/', 'α': 'alpha', '•': '.', 'à': 'a', '−': '-', 'β': 'beta', '∅': '', '³': '3', 'π': 'pi', }
    for p in punct_mapping:
        text = text.replace(p, punct_mapping[p])

    # 연속된 공백 치환
    text = re.sub(r"\s+", " ", text).strip()

    # 개행 문자 "\n" 제거
    text = text.replace('\n', '')

    return text

In [ ]:
import csv
from datetime import datetime

start_date = datetime.strptime('2023-07-15', '%Y-%m-%d').date()
end_date = datetime.strptime('2023-07-17', '%Y-%m-%d').date()  # 기간 설정
file_path = '/content/drive/MyDrive/이슈트래커/query_result.csv'
text = ''

with open(file_path, 'r', encoding='utf-8') as f:
    data = csv.reader(f, delimiter=',')
    header = next(data)
    for row in data:
        date_string = row[3]  # Assuming the date is in the second column (index 1)
        row_date = datetime.strptime(date_string, '%Y-%m-%d %H:%M:%S').date()
        if start_date <= row_date <= end_date:
           text += preprocessing(row[1])


In [ ]:
from konlpy.tag import Okt
import nltk

t = Okt()

text = text.replace('\n', '')

const_doc =text
tokens_const = t.nouns(const_doc)
tokens_const = [token for token in tokens_const if len(token) > 1]
stopwords = ['포토'] # 포토 제거
clean_tokens_const = []

for token in tokens_const:
  if token not in stopwords:
    clean_tokens_const.append(token)
f.close()

In [ ]:
const_cnt = {}
max_words = 20

for word in clean_tokens_const:
    const_cnt[word] = const_cnt.get(word, 0) + 1
sorted_w = sorted(const_cnt.items(), key=lambda kv: kv[1])
result = sorted_w[-max_words:]
result.reverse()
print(result)

[('올스타전', 27), ('올스타', 16), ('채은성', 15), ('우승', 10), ('롯데', 10), ('소크라테스', 8), ('감독', 8), ('홈런', 8), ('김민석', 7), ('사인', 6), ('퓨처', 6), ('기록', 6), ('구자욱', 5), ('홈런레이스', 5), ('싱글벙글', 5), ('수상', 5), ('승리', 5), ('김하성', 5), ('박세웅', 5), ('참가', 4)]


## 본문으로 키워드 추출

In [ ]:
import csv
from datetime import datetime

start_date = datetime.strptime('2023-07-15', '%Y-%m-%d').date()
end_date = datetime.strptime('2023-07-17', '%Y-%m-%d').date()  # 기간 설정
file_path = '/content/drive/MyDrive/이슈트래커/query_result.csv'
text = ''

with open(file_path, 'r', encoding='utf-8') as f:
    data = csv.reader(f, delimiter=',')
    header = next(data)
    for row in data:
        date_string = row[3]  # Assuming the date is in the second column (index 1)
        row_date = datetime.strptime(date_string, '%Y-%m-%d %H:%M:%S').date()
        if start_date <= row_date <= end_date:
           text += row[2]


In [ ]:
from konlpy.tag import Okt
import nltk

t = Okt()

text = text.replace('\n', '')

const_doc =text
tokens_const = t.nouns(const_doc)
tokens_const = [token for token in tokens_const if len(token) > 1]
stopwords = ['기자','보기','경기'] # 포토 제거
clean_tokens_const = []

for token in tokens_const:
  if token not in stopwords:
    clean_tokens_const.append(token)
f.close()

In [ ]:
const_cnt = {}
max_words = 20

for word in clean_tokens_const:
    const_cnt[word] = const_cnt.get(word, 0) + 1
sorted_w = sorted(const_cnt.items(), key=lambda kv: kv[1])
result = sorted_w[-max_words:]
result.reverse()
print(result)

[('올스타', 275), ('올스타전', 184), ('사직', 131), ('나눔', 131), ('부산', 129), ('신한은행', 128), ('현희', 126), ('사직야구장', 123), ('드림', 121), ('리그', 88), ('기록', 87), ('시즌', 73), ('홈런', 71), ('감독', 71), ('선수', 69), ('우승', 56), ('대회', 52), ('소년', 51), ('사진', 48), ('롯데', 46)]
